## Forked from https://www.kaggle.com/code/abdurrafae/improved-code-interpretation

**Lewis:** the only changes in this notebook are those needed to run the original one with the new Kaggle evaluation API

In [1]:
%reload_ext autoreload
%autoreload 2
import sys
print('python版本：',sys.version)

import pkg_resources

def get_package_version(package_name):
    try:
        version = pkg_resources.get_distribution(package_name).version
        return version
    except pkg_resources.DistributionNotFound:
        return "Package not found"
package_name = "torch"
version = get_package_version(package_name)
print(f"{package_name}版本：{version}")

import torch

cuda_version = torch.version.cuda
print("CUDA版本：", cuda_version)

python版本： 3.10.12 (main, Jul  5 2023, 18:54:27) [GCC 11.2.0]
torch版本：2.1.2


/tmp/ipykernel_12395/3843633427.py:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


CUDA版本： 12.1


In [2]:
import transformers
import xformers
transformers.__version__, xformers.__version__

('4.39.3', '0.0.23.post1')

In [4]:
import time

NOTEBOOK_START_TIME = time.time()

In [5]:
import aimo
try:
    env = aimo.make_env()
except:
    pass
iter_test = env.iter_test()

from utils import *

TO-DO

Change temperature as the question goes longer
Change temperature based on question lenght

# Zero-shot MMOS-DeepSeekMath-7B with self-consistency and generated code reasoning evaluation

Self-consistency is a modification of the standard greedy decoding in reasoning pipelines via sampling several diverse answers followed by aggregation, e.g., most common answer ([SC-CoT paper](https://arxiv.org/pdf/2203.11171.pdf)).

In this kernel, we will consider MMOS-DeepSeekMath-7B RL-tuned backbone; in my experiments, this model produces more consistent code reasoning and the code block execution will allow us to decrease arithmetic hallucinations.

In [6]:
DEBUG = False

QUANT = False

if QUANT:
    from transformers import BitsAndBytesConfig
    quantization_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

USE_PAST_KEY = True

In [7]:
%%time
if QUANT:
    !pip install -U ./input/accelerate-wheelwhl/accelerate-0.29.1-py3-none-any.whl -qq
    !pip install -U ./input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq


import torch
import gc
'''
accelerate execution a bit
'''
torch.backends.cuda.enable_mem_efficient_sdp(False)

from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    AutoConfig,
    StoppingCriteria,
    set_seed
)

import transformers
print(f"Transformers Version: {transformers.__version__}")
set_seed(42)

Transformers Version: 4.39.3
CPU times: user 30.5 ms, sys: 50.4 ms, total: 80.9 ms
Wall time: 80.6 ms


In [8]:
import pandas as pd
from tqdm import tqdm
PRIVATE = True

# df = pd.read_csv('./input/ai-mathematical-olympiad-prize/test.csv')
# df.head()

In [9]:
# def naive_parse(answer):
#     '''
#     get the first consecutive numbers
#     '''
#     out = []
#     start = False
#     end = False
#     answer = answer.split('answer')[-1]
#     for l in reversed(list(answer)):
#         if l in '0123456789' and not end:
#             start = True
#             out.append(l)
#         else:
#             if start:
#                 end = True
        
#     out = reversed(out)
#     return ''.join(out)

# import re
# import sys
# import subprocess

# def return_last_print(output, n):
#     lines = output.strip().split('\n')
#     if lines:
#         return lines[n]
#     else:
#         return ""

# def process_code(code, return_shell_output=False):
    
#     def repl(match):
#         if "real" not in match.group():
#             return "{}{}".format(match.group()[:-1], ', real=True)')
#         else:
#             return "{}{}".format(match.group()[:-1], ')')
#     code = re.sub(r"symbols\([^)]+\)", repl, code)

#     if return_shell_output:
#         code = code.replace('\n', '\n    ')
#             # Add a try...except block
#         code = "\ntry:\n    from sympy import *\n{}\nexcept Exception as e:\n    print(e)\n    print('FAIL')\n".format(code)
    
#     if not return_shell_output:
#         print(code)
#     with open('code.py', 'w') as fout:
#         fout.write(code)
    
#     batcmd = 'timeout 7 ' + sys.executable + ' code.py'
#     try:
#         shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
#         return_value = return_last_print(shell_output, -1)
#         print(shell_output)
#         if return_shell_output:
#             if return_value=='FAIL':
#                 CODE_STATUS = False
#                 return_value = return_last_print(shell_output, -2)
#                 if "not defined" in return_value:
#                     return_value+='\nTry checking the formatting and imports'
#             else:
#                 CODE_STATUS = True
#             return return_value, CODE_STATUS  
#         code_output = round(float(eval(return_value))) % 1000
#     except Exception as e:
#         print(e,'shell_output')
#         code_output = -1
    
#     if return_shell_output:
#         if code_output==-1:
#             CODE_STATUS = False
#         else:
#             CODE_STATUS = True
#         return code_output, CODE_STATUS  
    
    
#     return code_output


# def process_text_output(output):
#     result = output    
#     try:
#         result_output = re.findall(r'\\boxed\{(\d+)\}', result)

#         print('BOXED', result_output)
#         # if not len(result_output):
#         #     result_output = naive_parse(result) # this is toxic
#         # else:
#         result_output = result_output[-1]

#         print('BOXED FINAL', result_output)
#         if not len(result_output):
#             result_output = -1
        
#         else:
#             result_output = round(float(eval(result_output))) % 1000
    
#     except Exception as e:
#         print(e)
#         print('ERROR PARSING TEXT')
#         result_output = -1
    
#     return result_output


In [10]:
torch.cuda.empty_cache()
gc.collect()

0

In [11]:
import re
import math
import random

from collections import defaultdict

# n_repetitions = 12 if PRIVATE else 4 # Original notebook had 22 but times out :(
n_repetitions = 16 if PRIVATE else 4 # Original notebook had 22 but times out :(

TOTAL_TOKENS = 2048 # if PRIVATE else 512
# TOTAL_TOKENS = 512 # if PRIVATE else 512


if PRIVATE:
    TIME_LIMIT = 31500
else:
    TIME_LIMIT = 1

In [12]:
#list number of cuda
n_gpus = torch.cuda.device_count()
device_i = 0
if n_gpus > 1:
    device_i = 1



if PRIVATE:

    MODEL_PATH = "./input/deepseek-math"#"/kaggle/input/gemma/transformers/7b-it/1"
    DEEP = True

    config = AutoConfig.from_pretrained(MODEL_PATH)
    config.gradient_checkpointing = True

    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

    device_map = [('model.embed_tokens', 0),
                 ('model.layers.0', 0),
                 ('model.layers.1', 0),
                 ('model.layers.2', 0),
                 ('model.layers.3', 0),
                 ('model.layers.4', 0),
                 ('model.layers.5', 0),
                 ('model.layers.6', 0),
                 ('model.layers.7', 0),
                 ('model.layers.8', 0),
                 ('model.layers.9', 0),
                 ('model.layers.10', 0),
                 ('model.layers.11', 0),
                 ('model.layers.12', 0),
                 ('model.layers.13', 0),
                 ('model.layers.14', 0),
                 ('model.layers.15', 0),
                 ('model.layers.16', 0),
                 ('model.layers.17', 0),
                 ('model.layers.18', 0),
                 ('model.layers.19', 0),
                 ('model.layers.20', 0),
                 ('model.layers.21', 0),
                 ('model.layers.22', device_i),
                 ('model.layers.23', device_i),
                 ('model.layers.24', device_i),
                 ('model.layers.25', device_i),
                 ('model.layers.26', device_i),
                 ('model.layers.27', device_i),
                 ('model.layers.28', device_i),
                 ('model.layers.29', device_i),
                 ('model.norm', device_i),
                 ('lm_head', device_i)]

    device_map = {ii:jj for (ii,jj) in device_map}

    if QUANT:
        from transformers import BitsAndBytesConfig
        quantization_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            device_map="sequential",
            torch_dtype="auto",
            trust_remote_code=True, 
            quantization_config=quantization_config,
            config=config
        )
    else:  
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            device_map=device_map,
            torch_dtype="auto",
            trust_remote_code=True,
            #quantization_config=quantization_config,
            config=config
        )
    
    pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype='auto',
    device_map=device_map,
)
    from transformers import StoppingCriteriaList

    class StoppingCriteriaSub(StoppingCriteria):
        def __init__(self, stops = [], encounters=1):
            super().__init__()
            self.stops = [stop.to("cuda") for stop in stops]

        def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
            for stop in self.stops:
                last_token = input_ids[0][-len(stop):]
                if torch.all(torch.eq(stop,last_token)):
                    return True
            return False


    stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output"] #,  
    stop_words_ids = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]
    stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])
    
    model.dtype, model.hf_device_map

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [42]:
code = \
"""
Below is a math problem you are to solve (positive numerical answer):
\"{}\"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Assistant: 

Interesting, let's analyze step by step:""" # seems ok this one

cot = \
"""

Below is a math problem you are to solve (positive numerical answer!):
\"{}\"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.

Assistant:"""

prompt_options = [code,cot]


code_list = [
"""
Below is a math problem you are to solve (positive numerical answer):
\"{}\"

To accomplish this:
- List the variables and known values from the problem.
- Outline the steps using sympy to solve the problem:
  * Identify the sympy functions required for each step.
  * Implement these functions step-by-step, commenting each line for clarity.
- Calculate the final answer and verify it's a positive integer.
- Output the result within \\boxed{}.

Assistant:

Interesting, let's analyze step by step:""",

## official
# """
# \"{}\"

# Please reason step by step, and put your final answer within \\boxed{}.

# Assistant:""",

## math prompter
"""
Below is a math problem you are to solve (positive numerical answer):
"Each of the three-digits numbers $111$ to $999$ is coloured blue or yellow in such a way that the sum of any two (not necessarily different) yellow numbers is equal to a blue number. What is the maximum possible number of yellow numbers there can be?"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Assistant: 

Interesting, let's analyze step by step:

1. The maximum possible number of yellow numbers is when each blue number can be written as the sum of two yellow numbers in exactly one way.
2. The smallest blue number is $111+111=222$, and the largest is $999$.
3. The numbers from $111$ to $499$ are blue because they can be written as the sum of two copies of the same number.
4. The numbers from $500$ to $749$ are yellow because they can be written as the sum of two different numbers from $250$ to $499$.
5. The numbers from $750$ to $999$ are blue because they can be written as the sum of $250$ and one of the numbers from $500$ to $749$.
6. Therefore, the maximum number of yellow numbers is the number of numbers from $250$ to $499$, which is $250$.

So, the maximum possible number of yellow numbers there can be is $250$. The answer is: $250$

---

User:

Below is a math problem you are to solve (positive numerical answer):
\"{}\"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Assistant:

Interesting, let's analyze step by step:"""
# tuning2
"""
Below is a math problem you are to solve (natural number answer):
\"{}\"
To accomplish this, first list the problem's constraints, conditions and related knowledge, and thens solve the problem step by step. Be clear so even an idiot can follow your instructions. After solving the problem, output the final numerical answer within \\boxed{}.

Assistant:""",


# # original +
# """
# Below is a math problem you are to solve (positive numerical answer):
# \"{}\"
# To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
# Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

# Assistant: 

# Interesting, let's analyze step by step:""", # seems ok this one


]

all_prompt_options = [[code, code] for code in code_list]
# prompt_options = [code]


## Tree of Thoughts

In [ ]:
import os
from tree_of_thoughts import ToTAgent, MonteCarloSearch
from dotenv import load_dotenv
from swarms import Agent, HuggingfaceLLM

load_dotenv()

# Get the API key from the environment
api_key = os.environ.get("OPENAI_API_KEY")

# Initialize an agent from swarms
agent = Agent(
    agent_name="tree_of_thoughts",
    agent_description=(
        "This agent uses the tree_of_thoughts library to generate thoughts."
    ),
    system_prompt=None,
    llm=HuggingfaceLLM(
        "EleutherAI/gpt-neo-2.7B",
    ),
)

# Initialize the ToTAgent class with the API key
model = ToTAgent(
    agent,
    strategy="cot",
    evaluation_strategy="value",
    enable_react=True,
    k=3,
)


# Initialize the MonteCarloSearch class with the model
tree_of_thoughts = MonteCarloSearch(model)

# Define the initial prompt
initial_prompt = """


Input: 2 8 8 14
Possible next steps:
2 + 8 = 10 (left: 8 10 14)
8 / 2 = 4 (left: 4 8 14)
14 + 2 = 16 (left: 8 8 16)
2 * 8 = 16 (left: 8 14 16)
8 - 2 = 6 (left: 6 8 14)
14 - 8 = 6 (left: 2 6 8)
14 /  2 = 7 (left: 7 8 8)
14 - 2 = 12 (left: 8 8 12)
Input: use 4 numbers and basic arithmetic operations (+-*/) to obtain 24 in 1 equation
Possible next steps:
"""

# Define the number of thoughts to generate
num_thoughts = 1
max_steps = 3
max_states = 4
pruning_threshold = 0.5


# Generate the thoughts
solution = tree_of_thoughts.solve(
    initial_prompt=initial_prompt,
    num_thoughts=num_thoughts,
    max_steps=max_steps,
    max_states=max_states,
    pruning_threshold=pruning_threshold,
    # sleep_time=sleep_time
)

print(f"Solution: {solution}")

## Count Tokens

In [59]:
txt = \
"""
0_User:

Below is a math problem you are to solve (positive numerical answer):
"What is the minimum value of $5x^2+5y^2-8xy$ when $x$ and $y$ range over all real numbers such that $|x-2y| + |y-2x| = 40$?"

To accomplish this:
- List the variables and known values from the problem.
- Outline the steps using sympy to solve the problem:
  * Identify the sympy functions required for each step.
  * Implement these functions step-by-step, commenting each line for clarity.
- Calculate the final answer and verify it's a positive integer.
- Output the result within \boxed{}.

Assistant:

Interesting, let's analyze step by step:



Step 1: List the variables and known values from the problem.
The problem has two variables, $x$ and $y$. We are given the equation $|x-2y| + |y-2x| = 40$ and the function $f(x,y) = 5x^2+5y^2-8xy$ to minimize.

Step 2: Outline the steps using sympy to solve the problem.
- First, we need to use the given equation $|x-2y| + |y-2x| = 40$ to set up a constraint for the function $f(x,y)$.
- Then, we need to find the minimum value of $f(x,y)$ subject to the constraint.

Now, let's implement these steps:

Step 3: Identify the sympy functions required for each step.
We can use sympy to set up the constraint and find the minimum value of the function. Here's the code:

```python


INTERMEDIATE OUT :

from sympy import symbols, Eq, solve, diff, simplify

def solve_problem():
    x, y = symbols('x y')

    # Set up the constraint
    constraint = Eq(abs(x - 2*y) + abs(y - 2*x), 40)

    # Define the function to minimize
    f = 5*x**2 + 5*y**2 - 8*x*y

    # Find the critical points by setting the gradient equal to zero
    grad_f = [diff(f, var) for var in (x, y)]
    critical_points = solve(grad_f, (x, y))

    # Substitute the critical points into the constraint
    valid_points = [point for point in critical_points if simplify(constraint.subs({x: point[0], y: point[1]})) == True]

    # Evaluate the function at the valid points
    values = [f.subs({x: point[0], y: point[1]}) for point in valid_points]

    # Find the minimum value
    min_value = min(values)

    return min_value

result = solve_problem()
print(result)
```

CODE RESULTS Traceback: line 21, in <listcomp>
    valid_points = [point for point in critical_points if simplify(constraint.subs({x: point[0], y: point[1]})) == True]
TypeError: 'Symbol' object is not subscriptable

FAIL


INTERMEDIATE OUT :
```output
Traceback: line 21, in <listcomp>
    valid_points = [point for point in critical_points if simplify(constraint.subs({x: point[0], y: point[1]})) == True]
TypeError: 'Symbol' object is not subscriptable

FAIL

```
It seems there's a problem in the code above. We cannot directly substitute the values of $x$ and $y$ from the critical points into the constraint because we cannot subscript symbols with `subs` function. So, let's rewrite the code using a different approach:

Step 3: Identify the sympy functions required for each step.
- First, we'll define the constraint as a Sympy expression.
- Then, we'll define the function $f(x,y)$.
- We'll solve the constraint for $y$ in terms of $x$.
- We'll substitute this expression into the function to obtain a single-variable function.
- We'll find the critical points of this single-variable function by setting its derivative equal to zero.
- We'll evaluate the function at these critical points.
- We'll find the minimum value among these evaluations.

Now, let's implement these steps:

```python


INTERMEDIATE OUT :

from sympy import symbols, Eq, solve, diff, simplify, Abs

def solve_problem():
    x, y = symbols('x y')

    # Set up the constraint
    constraint = Eq(Abs(x - 2*y) + Abs(y - 2*x), 40)
    # Solve the constraint for y in terms of x
    y_expr = solve(constraint, y)[0]

    # Define the function to minimize
    f = 5*x**2 + 5*y_expr**2 - 8*x*y_expr

    # Find the derivative of the function with respect to x
    f_prime = diff(f, x)

    # Find the critical points by setting the derivative equal to zero
    critical_points = solve(f_prime, x)

    # Evaluate the function at the critical points
    values = [f.subs(x, point) for point in critical_points]

    # Find the minimum value
    min_value = min(values)

    return min_value

result = solve_problem()
print(result)
```

CODE RESULTS 800

INTERMEDIATE OUT :
```output
800
```
The minimum value of $5x^2+5y^2-8xy$ is $800$. But the problem asks for a positive integer, and $800$ is a positive integer. So the answer is $\boxed{800}$.
The minimum value of $5x^2+5y^2-8xy$ when $x$ and $y$ range over all real numbers such that $|x-2y| + |y-2x| = 40$ is 800.

So the minimum value of $5x^2 + 5y^2 - 8xy$ is $800$, which is a positive integer. The answer is: $800$."""

In [60]:


len(tokenizer(txt).input_ids)

1513